# Day 12
## Part 1


In [26]:
from dataclasses import dataclass
from collections import defaultdict, namedtuple
from functools import cached_property

@dataclass(eq=True, frozen=True)
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)

N = Point(0, 1)
S = Point(0, -1)
W = Point(-1, 0)
E = Point(1, 0)

DIRECTIONS = (N, E, S, W)

FenceSection = namedtuple("FenceSection", "location direction")

@dataclass(frozen=True)
class Region:
    plant: str
    area: frozenset

    @cached_property
    def perimeters(self):
        result = set()
        for p in self.area:
            for d in DIRECTIONS:
                if p + d not in self.area:
                    result.add(FenceSection(p, d)) 
        return result

    @cached_property
    def price(self):
        return len(self.perimeters) * len(self.area)

    @cached_property
    def fences(self):
        result = set()
        seen = set()
        for p in self.perimeters:
            if p not in seen:
                fence = {p}
                q = {p}
                seen.add(p)
                while q:
                    x = q.pop()
                    for nbr in self.perimeters - seen - fence:
                        # A pair of perimeter sections are neighbouring parts of a fence
                        # if the difference in location is 1 and they're facing in the
                        # same direction
                        if x.location - nbr.location in DIRECTIONS and x.direction == nbr.direction:
                            fence.add(nbr)
                            q.add(nbr)
                            seen.add(nbr)
                result.add(frozenset(fence))
        return result

    @cached_property
    def price_2(self):
        return len(self.fences) * len(self.area)
    

def parse_grid(s, convert=lambda x: x, ignore=""):
    grid = {}
    for y, line in enumerate(reversed(s.strip().splitlines())):
        for x, c in enumerate(line):
            if c not in ignore:
                grid[Point(x, y)] = convert(c)
    return grid

def parse_data(s):
    g = defaultdict(set)
    farm = parse_grid(s)
    for p in farm:
        for d in DIRECTIONS:
            nbr = p + d
            if farm.get(nbr, None) == farm[p]:
                g[p].add(nbr)

    seen = set()
    result = []
    for p in farm:
        if p not in seen:
            region = {p}
            q = {p}
            seen.add(p)
            while q:
                x = q.pop()
                for d in DIRECTIONS:
                    nbr = x + d
                    if farm[p] == farm.get(nbr, None) and nbr not in region | q:
                        region.add(nbr)
                        q.add(nbr)
                        seen.add(nbr)
            result.append(Region(farm[p], frozenset(region)))
    return result

def part_1(data):
    return sum(region.price for region in data)


test_data = parse_data("""RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE""")

assert part_1(test_data) == 1930

In [27]:
data = parse_data(open("input").read())
part_1(data)

1485656

## Part 2

In [29]:
def part_2(data):
    return sum(region.price_2 for region in data)

assert part_2(test_data) == 1206

In [30]:
part_2(data)

899196